Context: Quantum Teleportation Recap

    Alice wants to teleport the state of her qubit 0 to Bob's qubit 2.

    Alice and Bob share an entangled pair between qubits 1 and 2.

    Alice performs a Bell measurement on her qubits (0 and 1). This measurement collapses those qubits and gives her two classical bits of outcome — effectively telling her which Bell state the qubits collapsed into.

    Because of entanglement and measurement, Bob’s qubit 2 ends up in a state related to Alice's original qubit, but modified by some Pauli operators XX and ZZ depending on Alice’s measurement result.

| Alice’s result | Bob’s qubit state before correction | Correction needed | Bob’s final state matches Alice’s original |
| -------------- | ----------------------------------- | ----------------- | ------------------------------------------ |
| 00             | Exactly original                    | None              | Yes                                        |
| 01             | Original state with Z phase flip    | Apply $Z$         | Yes                                        |
| 10             | Original state with X bit flip      | Apply $X$         | Yes                                        |
| 11             | Original state with both flips      | Apply $X$ and $Z$ | Yes                                        |


In [ ]:
!pip install qiskit --quiet
!pip install qiskit_aer --quiet
!pip install pylatexenc --quiet

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
import IPython.display
import io
from PIL import Image

Qubit Roles

    Qubit 0: Alice’s input qubit — the qubit state you want to teleport.

    Qubit 1: Alice’s half of the entangled pair.

    Qubit 2: Bob’s half of the entangled pair — the qubit that will receive the teleported state.

In [ ]:
# Step 1: Create the full circuit with 3 qubits
qc = QuantumCircuit(3)  # 3 qubits, no classical bits since no measurement here

# Step 2: Alice's qubit (qubit 0) in a state to be teleported (e.g., Hadamard for superposition)
qc.h(0)  # You can replace this with other gates to test different states

# Step 3: Create entanglement between qubit 1 and 2 (shared between Alice and Bob)
qc.h(1)
qc.cx(1, 2)

# Step 4: Bell on Alice's side (qubits 0 and 1)
qc.cx(0, 1)
qc.h(0)

# So now qubits 1 and 2 form an EPR pair shared between Alice (qubit 1) and Bob (qubit 2).

# Step 5: Apply correction on Bob's qubit (qubit 2)
# We’ll use the Aer simulator’s "qasm_simulator" to simulate this
# and use the "conditional gates" in a second circuit later

qc.draw('mpl')

In [ ]:
# For execution in qiskit >= 1 (Newer version)

simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)

# Run with only 1 shot to extract a single measurement
result = simulator.run(compiled_circuit, shots=1, memory=True).result()
memory = result.get_memory()[0]  # e.g., '10'
print("Alice's measurement:", memory)


In [ ]:
# Create new circuit only for Bob's correction
correction = QuantumCircuit(3)

# Apply conditional corrections based on Alice's measurement
if memory[1] == '1':  # Remember: memory is little-endian ('cb')
    correction.x(2)
if memory[0] == '1':
    correction.z(2)


In [ ]:
# Remove measurements for pure state simulation
qc_no_measure = qc.remove_final_measurements(inplace=False)
full = qc_no_measure.compose(correction)

# Now simulate the state
final_state = Statevector.from_instruction(full)
bloch = plot_bloch_multivector(final_state.data, title="Bob's Qubit State")
buf = io.BytesIO()
bloch.savefig(buf)
buf.seek(0)
plt.close(bloch)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(Image.open(buf))
ax.axis('off')
ax.set_title('Teleport')
plt.show()


In [ ]:
from qiskit.quantum_info import partial_trace

# Get the reduced state for Bob's qubit (qubit 2)
bob_state = partial_trace(final_state, [0, 1])  # Trace out qubit 0 and 1

# Plot only Bob's Bloch sphere
bloch = plot_bloch_multivector(bob_state, title="Bob's Qubit After Teleportation")
buf = io.BytesIO()
bloch.savefig(buf)
buf.seek(0)
plt.close(bloch)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.imshow(Image.open(buf))
ax.axis('off')
ax.set_title('Teleportation Result (Bob)')
plt.show()